In [ ]:
with open('/home/cqm/datasets/celeba/index.txt') as input_file:
    labels = []
    images = []
    for line in input_file.readlines():
        line = line.strip('\n').split(" ")
        imgid = line[0]
        for i in range(1, len(line)):
            labels.append(imgid)
            images.append(line[i])
print('images and labels prepared')  

In [ ]:
print labels[400*128]

In [ ]:
with open('/home/cqm/datasets/celeba/index.txt') as input_file:
    labels = []
    images = []
    labels_b = []
    images_b = []
    count1 = 0
    count2 = 0
    for line in input_file.readlines():
        line = line.strip('\n').split(" ")
        imgid = line[0]
        for i in range(1, len(line)):
            if count1 >= 128:
                count1 = 0 
            if count1 < 64 and count2%2 == 0:
                labels.append(imgid)
                images.append(line[i])
                count1 += 1
                count2 += 1
            elif count1 < 64 and count2%2 == 1:
                labels_b.append(imgid)
                images_b.append(line[i])
                count1 += 1
                count2 += 1
            elif count1 < 96:
                labels.append(imgid)
                images.append(line[i])
                count1 += 1
                count2 += 1 
            elif count1 < 128:
                labels_b.append(imgid)
                images_b.append(line[i])
                count1 += 1
                count2 += 1
print('images and labels prepared')
print len(labels), len(images),len(labels_b), len(images_b)

In [ ]:
print labels[400*64]
3063 016870.jpg 073859.jpg 158145.jpg
3064 165103.jpg 169435.jpg 172406.jpg 174072.jpg 178397.jpg 178857.jpg
3065 018685.jpg 020421.jpg 030522.jpg 035702.jpg 039798.jpg 102418.jpg 107034.jpg 128058.jpg 156975.jpg 158330.jpg
3066 004580.jpg 006779.jpg 032653.jpg 032665.jpg 039101.jpg 047802.jpg 052432.jpg 055288.jpg 057025.jpg 057347.jpg 062589.jpg 065327.jpg 069574.jpg 108118.jpg 113621.jpg 132199.jpg 133261.jpg 135264.jpg 135780.jpg 147598.jpg 147706.jpg 155663.jpg

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

x = tf.placeholder(tf.float32,[None, 64, 64, 3])
y = tf.placeholder(tf.float32, [None])
#y_is_same = tf.placeholder(tf.float32, [None])
keep_prob = tf.placeholder(tf.float32)
batch_size = 64


with open('/home/cqm/datasets/celeba/index.txt') as input_file:
    person_index = []
    for line in input_file.readlines():
        line = line.strip('\n').split(" ")
        person_index.append(line[1:])
    print "image and label data loaded"
    
path = '/home/cqm/datasets/celeba/img/'  

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides = [1, 1, 1, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

def network(x):
    w_conv1 = weight_variable([3, 3, 3, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x, w_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1) #32*32*32

    w_conv2 = weight_variable([3, 3, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2) #64*16*16

    w_conv3 = weight_variable([3, 3, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, w_conv3) + b_conv3)
    h_pool3 = max_pool_2x2(h_conv3) #128*8*8


    h_pool3_reshape = tf.reshape(h_pool3, [-1, 8*16*64])

    w_fc1 = weight_variable([8*16*64, 512])
    b_fc1 = bias_variable([512])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_reshape, w_fc1) + b_fc1)
    return h_fc1

def contrastive_loss(y, d):
    tmp = y*tf.square(d)
    tmp2 = (1-y)*tf.square(tf.maximum((20-d), 0))
    return tf.reduce_sum(tmp+tmp2)/batch_size

def getface(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('/home/cqm/datasets/celeba/haarcascade_frontalface_default.xml')
    face = face_cascade.detectMultiScale(gray, 1.3, 4)
    if len(face) > 0:
        for x, y, w, h in face:
            continue
        img = img[y:y+h, x:x+w]
    img = cv2.resize(img, (64, 64))/127.5-1.0
    return img
        

out = network(x)
d = tf.reduce_sum((out[:batch_size] - out[batch_size:])**2+1e-5, axis = 1)**0.5

loss = contrastive_loss(y, d)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
#correct_prediction = tf.equal(tf.argmax(y_out,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
#saver = tf.train.Saver(max_to_keep = 1)

for i in range(50000):
    pos_pair_list1 = []
    pos_pair_list2 = []
    while len(pos_pair_list1) < 32:
        id = np.random.choice(len(person_index),1,replace=False)[0]
        if len(person_index[id]) >= 2:
            pair_id = np.random.choice(len(person_index[id]),2,replace=False)
            pos_pair_list1.append(person_index[id][pair_id[0]])
            pos_pair_list2.append(person_index[id][pair_id[1]])

    neg_pair_list1 = []
    neg_pair_list2 = []
    while len(neg_pair_list1) < 32:
        id = np.random.choice(len(person_index),2,replace=False)
        neg_id1 = np.random.choice(len(person_index[id[0]]),1,replace=False)[0]
        neg_id2 = np.random.choice(len(person_index[id[1]]),1,replace=False)[0]
        neg_pair_list1.append(person_index[id[0]][neg_id1])
        neg_pair_list2.append(person_index[id[1]][neg_id2])
    
    x_imgs = []
    y_labels = [1] * 32 + [0] * 32
    for img_name in pos_pair_list1:
        img = cv2.imread(path + img_name)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = cv2.resize(img, (64, 64))/127.5 - 1.0
        img = getface(img)
        x_imgs.append(img)
    for img_name in neg_pair_list1:
        img = cv2.imread(path + img_name)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = cv2.resize(img, (64, 64))/127.5 - 1.0
        img = getface(img)
        x_imgs.append(img)
    for img_name in pos_pair_list2:
        img = cv2.imread(path + img_name)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = cv2.resize(img, (64, 64))/127.5 - 1.0
        img = getface(img)
        x_imgs.append(img)
    for img_name in neg_pair_list2:
        img = cv2.imread(path + img_name)        
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #img = cv2.resize(img, (64, 64))/127.5 - 1.0
        img = getface(img)
        x_imgs.append(img)
    x_imgs = np.array(x_imgs, dtype=np.float32)

    if i%10 == 0:
        train_loss = loss.eval(feed_dict={x:x_imgs,y:y_labels})
        print('step %d , training loss %f'%(i, train_loss))
    train_step.run(feed_dict={x:x_imgs,y:y_labels})

In [ ]:
import cv2
import os

path = 'test/3/'
dir_list = os.listdir(path)

for imgid in dir_list:
    img = cv2.imread(path + imgid)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('/home/cqm/datasets/celeba/haarcascade_frontalface_default.xml')
    face = face_cascade.detectMultiScale(gray, 1.3, 4)
    print len(face)
    if len(face) > 0:
        for x, y, w, h in face:
            continue
        img = img[y:y+h, x:x+w]
    img = cv2.resize(img,(64,64))
    cv2.imwrite(path+imgid, img)

In [ ]:
from matplotlib import pyplot as plt
person = cv2.imread('test/409.jpg')
gray = cv2.cvtColor(person, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier('/home/cqm/datasets/celeba/haarcascade_frontalface_default.xml')
face = face_cascade.detectMultiScale(gray, 1.3, 4)
print len(face)
if len(face) > 0:
    for x, y, w, h in face:
        continue
    cv2.rectangle(person, (x, y), (x+w,y+h),(0,255, 0),2)
plt.imshow(person[:,:,::-1])
plt.show()

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

x = tf.placeholder(tf.float32,[None, 64, 64, 3])
y = tf.placeholder(tf.float32, [None])
yid = tf.placeholder(tf.float32, [None])
keep_prob = tf.placeholder(tf.float32)
batch_size = 64


with open('/home/cqm/datasets/celeba/index.txt') as input_file:
    person_index = []
    label_index = []
    for line in input_file.readlines():
        line = line.strip('\n').split(" ")
        label_index.append(line[0])
        person_index.append(line[1:])
    print "image and label data loaded"
    
path = '/home/cqm/datasets/celeba/img/'  

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, w):
    return tf.nn.conv2d(x, w, strides = [1, 1, 1, 1], padding = "SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

def network(x):
    w_conv1 = weight_variable([3, 3, 3, 32])
    b_conv1 = bias_variable([32])
    h_conv1 = tf.nn.relu(conv2d(x, w_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1) #32*32*32

    w_conv2 = weight_variable([3, 3, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2) #64*16*16

    w_conv3 = weight_variable([3, 3, 64, 128])
    b_conv3 = bias_variable([128])
    h_conv3 = tf.nn.relu(conv2d(h_pool2, w_conv3) + b_conv3)
    h_pool3 = max_pool_2x2(h_conv3) #128*8*8


    h_pool3_reshape = tf.reshape(h_pool3, [-1, 8*16*64])

    w_fc1 = weight_variable([8*16*64, 512])
    b_fc1 = bias_variable([512])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_reshape, w_fc1) + b_fc1)
    return h_fc1

def contrastive_loss(y, d):
    tmp = y*tf.square(d)
    tmp2 = (1-y)*tf.square(tf.maximum((20-d), 0))
    return tf.reduce_sum(tmp+tmp2)/batch_size
        

x_out = network(x)
d_pos = tf.reduce_sum(tf.square(x_out[:64] - x_out[64:128]), 1)
d_neg = tf.reduce_sum(tf.square(x_out[:64] - x_out[128:]), 1)
loss = tf.reduce_mean(tf.maximum(0.0, 50 + d_pos - d_neg))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
#correct_prediction = tf.equal(tf.argmax(y_out,1), tf.argmax(y_,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep = 1)

for i in range(20000):
    pos_pair_list1 = []
    pos_pair_list2 = []
    neg_pair_list = []
    while len(pos_pair_list1) < 64:
        id = np.random.choice(len(person_index),2,replace=False)
        if len(person_index[id[0]]) >= 2:
            pair_id = np.random.choice(len(person_index[id[0]]),2,replace=False)
            neg_id = np.random.choice(len(person_index[id[1]]),1,replace=False)[0]
            pos_pair_list1.append(person_index[id[0]][pair_id[0]])
            pos_pair_list2.append(person_index[id[0]][pair_id[1]])
            neg_pair_list.append(person_index[id[1]][neg_id])
    x_imgs = []
    for img_name in pos_pair_list1:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        #img = getface(img)
        x_imgs.append(img)
    for img_name in pos_pair_list2:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        #img = getface(img)
        x_imgs.append(img)
    for img_name in neg_pair_list:
        img = cv2.imread(path + img_name)
        img = cv2.resize(img, (64, 64))/127.5 - 1.0
        #img = getface(img)
        x_imgs.append(img)
    x_imgs = np.array(x_imgs, dtype=np.float32)

    if i%10 == 0:
        train_loss = loss.eval(feed_dict={x:x_imgs})
        print('step %d , training loss %f'%(i, train_loss))
    train_step.run(feed_dict={x:x_imgs})
    
    
saver.save(sess,'model/face2.ckpt')

In [ ]:
import cv2
test1 = cv2.imread('testimg/eita1.jpg')
test1 = cv2.resize(test1,(64, 64))
test2 = cv2.imread('testimg/eita2.jpg')
test2 = cv2.resize(test2,(64, 64))
test3 = cv2.imread('testimg/long1.jpg')
test3 = cv2.resize(test3,(64, 64))


In [ ]:
test1 = test1.astype('float32')[None]/127.5 - 1.0
test2 = test2.astype('float32')[None]/127.5 - 1.0
test3 = test3.astype('float32')[None]/127.5 - 1.0

In [ ]:
res1 = x_out.eval(feed_dict={x:test1})
res2 = x_out.eval(feed_dict={x:test2})
res3 = x_out.eval(feed_dict={x:test3})

In [ ]:
d1 = ((res1-res2)**2).sum()
d2 = ((res1-res3)**2).sum()

In [ ]:
print d1
print d2

In [ ]:
res1